In [ ]:
'''
Implement the Mott-Smith Langmuir model of spacecraft charging
See "Spacecraft Charging" (article) Shu T. Lai, Kerri Cahoy, and my notes from Adv. Space Physics 

J_e,0 (1 - < δ(E) + η(E) >) exp( -qφ/(k_B T) ) = J_i,0 (1 - qφ/(k_B T) )^alpha

where:
    J_e,0 is the flux of electrons: J_e,0 = 2 \pi q_e \int_0^\infty f(E) E dE, where q_e<0 is e- charge, E=energy, and f(E) is the e- omnidirectional energy distribution
    J_i,0 is the flux of ions: J_i,0 = q_i * n_i * v_i,   q_i>0 is ion charge, n_i is ion density, v_i is ion bulk speed
    q<0 is the elementary charge (sign?) 
    T is the temperature  (species-dependent?)
    δ(E) is the secondary electron rate (a function of energy E) --- i.e. the number of emitted electrons per incident electron with energy E
    η(E) is the backscattering rate (a function of energy E) --- i.e. the number of backscattered electrons per incident electron with energy E
    φ is the spacecraft potential

The idea is to solve for φ given the other parameters
'''

import numpy as np
from numpy.polynomial.laguerre import laggauss
from scipy.optimize import fsolve


def f(E, n, T):
    '''
    calculate the electron "Mawellian" (exponential) distribution f(E)   [E,n,T,f all in SI units]
    '''
    k_B = 1.380649e-23
    m_e = 9.10938e-31
    f = n * (m_e / (2 * np.pi * k_B * T))**1.5 * np.exp(-E / (k_B * T))


def delta(E, material = "dummy"):
    '''
    Secondary electron spectrum delta(E), for a given material
    See e.g. Lin and Joy '05 to get empirical functions for different materials
    Inputs:
        E [Joules]
        material: string denoting the material of the spacecraft
    '''
    if material == "dummy":
        return np.exp(-E)  #0.5


def eta(E, material = "dummy"):
        '''
    Backscattered electron spectrum delta(E), for a given material
    Inputs:
        E [Joules]
        material: string denoting the material of the spacecraft
    '''
    if material == "dummy":
        return 0.
    

def weighted_avg(func, npts=5):
    '''
    Assuming that fmax is a Maxwellian wrt velocity, exponential wrt energy: fmax(E)= exp(-E)
    perform weighted average < func(E) > = \int_0^\infty fmax(E) func(E) E dE / \int_0^\infty fmax(E) E dE
    using Gauss-Laguerre quadrature (esp. suitable for integrated product of exponential with another function )
        func: the function func(E) to be integrated
        npts: the number of sample points
            These sample points and weights will exactly integrate polynomials of degree  2*npts-1 or less
            or approximately integrate arbitrary functions that are well approximated by polynomials of that degree
    '''
    sample_E, weights = laggauss(npts)
    # approximate the integrals w/ Gauss-Laguerre quadrature and divide them:
    return np.nansum(weights * func(sample_E) * sample_E ) /  np.nansum(weights * sample_E )   



def phi_msl(n, T, v, material = "dummy", alpha = 0.):
    '''
    Calculate spacecraft potential phi from Mott-Smith Langmuir model
    Requires an iterative solver (e.g. Newton's Method)
        Inputs:
        n: density [SI]
        T: temperature [SI]   --> DIFFERENT TEMPERATURE between protons and electrons??
        v: bulk speed of ions [SI]
        material: the material of the spacecraft surface
        alpha: a parameter related to the spacecraft geometry. alpha = 0 for an infinite plane
    '''
    # calculate <delta + eta>
    #   delta and eta are likely to be given in the literature as functions of energy [e.g., in Joules].
    #   but in the integrals used in weighted_avg, the integration variable E is dimensionless.
    #   need to multiply times k_B T in the lambda expression below to account for this.
    k_B = 1.380649e-23
    q = -1.602176e-19        # SIGN??
    delta_eta = lambda en: delta(en*k_B*T, material = material) +  eta(en*k_B*T, material = material)
    delta_eta_ave = weighted_avg(delta_eta, npts=5)
    # solve for the potential that balances the currents
    def msl_eq(phi, J_e0, J_i0, T, alpha):
        return J_e0 * (1 - delta_eta_ave) * np.exp( -q* phi/(k_B * T) ) - J_i0 (1 - q * phi/(k_B * T) )**alpha
    phi_guess = 5.
    J_e0 = 
    J_i0 =  
    args = (J_e0, J_i0, T, alpha) 
    phi_solution = fsolve(msl_eq, phi_guess, args = args)
    return phi_solution



In [17]:
material = "dummy"
delta_eta = lambda en: delta(en, material = material) +  eta(en, material = material)
delta_eta_ave = weighted_avg(delta_eta, npts=66)
delta_eta_ave

0.2500000000000032